In [ ]:
from datetime import timedelta
from thenthis.widget import BaseModelWidget, ListWidget, DictionaryWidget
import thenthis.configs as configs
from ipywidgets import widgets
from IPython.display import display
import whendo.core.util as util

In [ ]:
cfg = configs.pdx_config

In [ ]:
cfg.show_servers()

In [ ]:
cfg.show_info()

In [ ]:
cfg.show_pin_states()

In [ ]:
cfg.initialize_servers()

In [ ]:
util.PP.pprint(
    cfg.schedule_programs(
        start_stop=util.DateTime2(
            dt1=util.Now.dt(), dt2=util.Now.dt() + timedelta(seconds=30)
        )
    )
)

In [ ]:
cfg.schedule_toggles(
    start_stop=util.DateTime2(
        dt1=util.Now.dt(), dt2=util.Now.dt() + timedelta(hours=8)
    )
)

In [ ]:
util.PP.pprint(
    cfg.schedule_notifications()
)

In [ ]:
util.PP.pprint(
    cfg.unschedule_pivots()
)

In [ ]:
tab_contents = ["actions", "schedulers", "programs", "servers"]
children = [
    DictionaryWidget(dictionary=dictionary).ui()
    for dictionary in [cfg.actions, cfg.schedulers, cfg.programs, cfg.servers]
]
tab = widgets.Tab(children=children, titles=tab_contents)

In [ ]:
display(tab)

In [ ]:
util.PP.pprint(cfg.server_client(cfg.pi4).load_dispatcher().dict())

display(DictionaryWidget(dictionary=cfg.server_client(cfg.pi4).load_dispatcher().dict()).ui())

display(BaseModelWidget(model=cfg.server_client(cfg.pi4).load_dispatcher()).ui())